# Part I. Data cleaning

In [1]:
#import packages
import pandas as pd
import numpy as np

## User data

In [2]:
user_data = pd.read_csv('data_format1/user_log_format1.csv')
user_data.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [3]:
user_data.shape

(54925330, 7)

In [4]:
#find out the missing value in this file
user_data.isnull().sum()

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64

In [5]:
#get the mode brand id for all stores, fillin NA value in brain_id column
missing = user_data[user_data.brand_id.isnull()].index
seller = user_data.groupby(['seller_id']).apply(lambda x:x.brand_id.mode()[0]).reset_index()
get_brand = user_data.loc[missing]
get_brand = get_brand.merge(seller,how='left',on=['seller_id'])[0].astype('float32')
get_brand.index = missing
user_data.loc[missing,'brand_id'] = get_brand

In [6]:
#see whether null values are filled
user_data.isnull().sum()

user_id        0
item_id        0
cat_id         0
seller_id      0
brand_id       0
time_stamp     0
action_type    0
dtype: int64

In [7]:
#unique customer purchase history
user_data['user_id'].nunique()

424170

In [8]:
user_data.shape

(54925330, 7)

In [9]:
user_data.to_csv('cleaned_raw/user_data.csv')

## User info

In [10]:
#user info
user_info = pd.read_csv('data_format1/user_info_format1.csv')
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [11]:
#user info
user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

In [12]:
user_info.age_range.fillna(user_info.age_range.median(),inplace=True)
user_info.gender.fillna(2,inplace=True)
#user info NA value exam
user_info.isnull().sum()

user_id      0
age_range    0
gender       0
dtype: int64

In [13]:
user_info = user_info[user_info['age_range'] != 0]

In [14]:
df_age = pd.get_dummies(user_info.age_range,prefix='age')
df_gender = pd.get_dummies(user_info.gender)
df_gender.rename(columns={0:'female',1:'male',2:'unknown'},inplace=True)
user_info = pd.concat([user_info.user_id, df_age, df_gender], axis=1)
del df_age, df_gender

In [15]:
user_info.to_csv('cleaned_raw/user_info.csv')